#### OBU (海外帳戶) 資料串聯

OBU篩選條件: 
* 檔案DW_CSCUST_SS: 欄位'CUST_TYP'(基本資料性質別) == '1Z', '2Z', '3Z', '4Z', '5Z', '6Z', '7Z'
* 檔案DW_AMLBRN_CUST: 欄位"ACTIVE_CUST_0024_FG" == "Y"

#### Module Import

In [2]:
# Import required package
import xlsxwriter
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import os
import numpy as np

#### 使用Python read_sql功能

##### Testing SQL Command (測試是否成功與JupyterLab連結)

In [3]:
# EY補充
# 下面程式碼為EY以外部人員身分連結至JupyterLab執行之程式碼測試


# Read data from db
# Make sure this block was executed successfully before moving to the following blocks.
# initilize DB connection
server = '192.168.53.240,31017' #server若有更改，需要更新
database = 'CRR93601' #DB若有更改，需要更新
username = 'BIGDATA93601' #使用者名稱若有更改，需要更新
password = 'M@egaicbc99!' #密碼若有替換，需要更新

# 連接JupyterLab
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)

# 測試SQL指令
query = '''
    SELECT TOP 10 CYC_MN FROM DW_CSCUST_SS;
'''

data = pd.read_sql(query, conn)

# close the DB connection
conn.close()


print('complete')

complete


##### OBU Join

In [4]:
# EY補充
# 下面程式碼為EY以外部人員身分連結至JupyterLab執行之程式碼測試
# 若擁有SQL權限則僅需要使用下面query中之SQL Command

# Read data from db
# Make sure this block was executed successfully before moving to the following blocks.
# initilize DB connection
server = '192.168.53.240,31017' #server若有更改，需要更新
database = 'CRR93601' #DB若有更改，需要更新
username = 'BIGDATA93601' #使用者名稱若有更改，需要更新
password = 'M@egaicbc99!' #密碼若有替換，需要更新

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)



# EY補充
# query中使用SQL語法
# 下面針對步驟進行相關說明
# 
# 1. (SELECT * FROM DW_CSCUST_SS WHERE ((CUST_TYP IN ('1Z', '2Z', '3Z', '4Z', '5Z', '6Z', '7Z')) AND (CYC_MN BETWEEN '2019-09-01' AND '2020-08-30')))
# 從資料DW_CSCUST_SS中篩選欄位CUST_TYP為1Z~7Z且年份區間為2019-09-01至2020-08-01的資料
#
# 2. JOIN (SELECT * FROM DW_AMLBRN_CUST WHERE CUST_ID != '' AND ACTIVE_CUST_0024_FG = 'Y') ON a.CYC_MN = b.CYC_MN AND a.CUST_ID = b.CUST_ID AND a.CUST_DUP_NO = b.CUST_DUP_NO
# 從資料DW_AMLBRN_CUST中篩選CUST_ID不為空值且ACTIVE_CUST_0024_FG為Y之資料並與步驟一篩選出的資料進行INNER JOIN
# 步驟1與步驟2主要目的為找出OBU客戶之ID 且 決定需要的資料年份區間
# 
# 3. 後續的程式碼則是決定需要哪些檔案並透過LEFT JOIN的方式確保僅篩選OBU客戶資料
#
# 若需要串聯DBU個人及DBU法人，則需要將步驟1 & 步驟2調整為其篩選條件
# 若需要調整資料年份區間，則需要調整步驟1之年份區間


query = '''
SELECT *
  FROM (SELECT * FROM DW_CSCUST_SS 
 WHERE ((CUST_TYP IN ('1Z', '2Z', '3Z', '4Z', '5Z', '6Z', '7Z')) AND (CYC_MN BETWEEN '2019-09-01' AND '2020-08-30'))) AS a
     
       JOIN (SELECT * FROM DW_AMLBRN_CUST WHERE CUST_ID != '' AND ACTIVE_CUST_0024_FG = 'Y') AS b 
       ON a.CYC_MN = b.CYC_MN AND a.CUST_ID = b.CUST_ID AND a.CUST_DUP_NO = b.CUST_DUP_NO
     
       LEFT OUTER JOIN (SELECT * FROM DW_AMLCUST_SS WHERE CUST_ID != '') AS c 
       ON a.CYC_MN = c.CYC_MN AND a.CUST_ID = c.CUST_ID AND a.CUST_DUP_NO = c.CUST_DUP_NO
     
       LEFT OUTER JOIN (SELECT * FROM DW_CMFLUREL_SS WHERE ID_NO != '') AS d 
       ON a.CYC_MN = d.CYC_MN AND a.CUST_ID = d.ID_NO AND a.CUST_DUP_NO = d.ID_DUP
     
       LEFT OUTER JOIN (SELECT * FROM ODS_CMFCUS25_SS WHERE CMFCUS25_ID_NO != '') AS e 
       ON a.CYC_MN = e.CYC_MN AND a.CUST_ID = e.CMFCUS25_ID_NO AND (a.CUST_DUP_NO = e.CMFCUS25_DUP_NO OR (a.CUST_DUP_NO = '' AND e.CMFCUS25_DUP_NO = '0'))
     
       LEFT OUTER JOIN (SELECT * FROM ODS_CMFCUS1_SS WHERE CMFCUS1_ID_NO != '') AS f 
       ON a.CYC_MN = f.CYC_MN AND a.CUST_ID = f.CMFCUS1_ID_NO AND (a.CUST_DUP_NO = f.CMFCUS1_DUP_NO OR (a.CUST_DUP_NO = '' AND f.CMFCUS1_DUP_NO = '0'))
     
       LEFT OUTER JOIN (SELECT * FROM DW_AMLCRR WHERE CUST_ID != '') AS g 
       ON a.CYC_MN = g.CYC_MN AND a.CUST_ID = g.CUST_ID AND a.CUST_DUP_NO = g.CUST_DUP_NO
'''

data = pd.read_sql(query, conn)

# close the DB connection
conn.close()
print('complete')


complete


In [ ]:
# 查看資料之Row & Column數量
print(data.shape)

In [ ]:
# 儲存資料至相關路徑
data.to_csv('/path', index=False) #路徑需更新